In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

In [14]:
from wordvecs import WordVectors, EmbeddingLayer

wordvectors = WordVectors(fname="/data/matthew/GoogleNews-vectors-negative300.bin", negvectors=False)

In [15]:
from sentiment_sents import Sentiment

# just load the sentences from the CNN system
sentiment = Sentiment("prevwork/CNN_sentence/")

In [16]:
len(sentiment.test_X), len(sentiment.train_X)

(2132, 8530)

In [17]:
len(wordvectors.vectors)

3000000

In [18]:
for w in sentiment.test_X:
    wordvectors.tokenize(w)
for w in sentiment.train_X:
    wordvectors.tokenize(w)

In [19]:
len(wordvectors.word_location)

21418

In [20]:
class SentimentExp(object):
    
    def __init__(self, train_X, train_Y, wordvecs=wordvectors):
        self.train_X = train_X
        self.train_Y = train_Y
        self.wordvecs = wordvecs
        
        self.input_size = 10  # not used
        self.batch_size = 50
        
        self.learning_rate = .01
        self.momentum = .9
        
        #self.train_X_rep = np.array([[self.getRep(x)] for x in self.train_X])
        self.train_X_rep = np.array([wordvecs.tokenize(x) for x in self.train_X])
        
        self._setup()

    def getRep(self, sent):
        ret = []
        for i in xrange(self.input_size):
            if i < len(sent):
                ret.append(self.wordvecs[sent[i]])
            else:
                ret.append(np.zeros(self.wordvecs.vector_size))
        return np.matrix(ret).reshape((1, self.input_size, self.wordvecs.vector_size))

    def _setup(self):
        self.x_batch = T.imatrix('x')
        self.y_batch = T.ivector('y')
        
        self.input_l = lasagne.layers.InputLayer((None, 50))
        
        self.embedding_l = EmbeddingLayer(
            self.input_l,
            W=self.wordvecs.get_numpy_matrix(),
            add_word_params=True,
        ) 
        
        self.first_l = lasagne.layers.Conv2DLayer(
            self.embedding_l,
            num_filters=100,
            filter_size=(2, self.wordvecs.vector_size),
            name='conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.first_l_max = lasagne.layers.MaxPool2DLayer(
            self.first_l,
            pool_size=(1,9)
        )
        
        self.hidden1_l = lasagne.layers.DenseLayer(
            self.first_l_max,
            num_units=50,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.hidden1_l_drop = lasagne.layers.DropoutLayer(
            self.hidden1_l,
            p=.25,
        )
                
        self.out_l = lasagne.layers.DenseLayer(
            self.hidden1_l_drop,
            num_units=1,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.output = lasagne.layers.get_output(self.out_l, self.x_batch)
        
        self.loss_vec_old = (self.output.reshape((self.output.size,)) - self.y_batch) ** 2
        self.output_diff = T.neq((self.output.flatten() > .5),(self.y_batch > .5)).sum()
        self.loss_vec = lasagne.objectives.binary_crossentropy(T.clip(self.output.reshape((self.output.size,)), .01, .99), self.y_batch)
        
        self.all_params = lasagne.layers.get_all_params(self.out_l)
        
        self.updates = lasagne.updates.adagrad(self.loss_vec.mean(), self.all_params, .01)
        #self.updates = lasagne.updates.apply_momentum(self.updates_adagrad)
        
        self.train_func = theano.function(
            [self.x_batch, self.y_batch],
            [self.loss_vec.mean(), self.loss_vec],
            updates=self.updates,
        )
        
        self.loss_func = theano.function(
            [self.x_batch, self.y_batch],
            [self.loss_vec.sum(), self.loss_vec, self.output_diff],
        )
        
    def _make_zero(self):
        self.embedding_l.W.get_value(borrow=True)[0,:] = 0
        
    def train(self):
        for s in xrange(0, len(self.train_X_rep), self.batch_size):
            end = s + self.batch_size
            if end > len(self.train_X_rep):
                end = len(self.train_X_rep)
            X_vals = np.array(self.train_X_rep[s:end]).astype('int32')
            y_vals = np.array(self.train_Y[s:end]).astype('int32')
            loss, _ = self.train_func(X_vals, y_vals)
            self._make_zero()
            
    def test_loss(self, test_X, test_Y):
        test_X_rep = np.array([self.wordvecs.tokenize(x) for x in test_X])
        loss_sum = 0.0
        wrong = 0.0
        for s in xrange(0, len(test_X_rep), self.batch_size):
            end = s + self.batch_size
            if end > len(self.train_X_rep):
                end = len(self.train_X_rep)
            X_vals = np.array(self.train_X_rep[s:end]).astype('int32')
            y_vals = np.array(self.train_Y[s:end]).astype('int32')
            loss, _, output_diff = self.loss_func(X_vals, y_vals)
            wrong += output_diff
            loss_sum += loss
        return loss_sum / len(test_X_rep), wrong / len(test_X_rep)
    
experiment = SentimentExp(sentiment.train_X, sentiment.train_Y)

In [ ]:
experiment.all_params

[Embeddings, conv1.W, conv1.b, W, b, W, b]

In [ ]:
for i in xrange(900):
    if i % 15 == 0:
        print i, experiment.test_loss(sentiment.train_X, sentiment.train_Y), experiment.test_loss(sentiment.test_X, sentiment.test_Y)
    experiment.train()

0 (2.1815189209238288, 0.49894490035169986) 

In [ ]:
experiment.test_loss(sentiment.train_X, sentiment.train_Y)

In [ ]:
experiment.test_loss(sentiment.test_X, sentiment.test_Y)

In [ ]:
test_func = theano.function(
    [experiment.x_batch, experiment.y_batch],
    [experiment.loss_vec.mean(), experiment.loss_vec, experiment.output, 
     T.grad(experiment.loss_vec.mean(), experiment.out_l.get_params()[0]),
     experiment.out_l.get_params()[0], experiment.y_batch, 
     #lasagne.layers.get_output(experiment.first_l, experiment.x_batch)
    ]
)

In [ ]:
test_func(np.array(experiment.train_X_rep[0:10]),np.array(experiment.train_Y[0:10]).astype('int32'))

In [ ]:
gg_k = theano.shared(0.)
gg_i = T.iscalar('x')

gg_res, gg_update = theano.scan(lambda: {gg_k:(gg_k + 1)}, sequences=[range(gg_i)])

In [ ]:
gg_update

In [ ]:
a1 = T.matrix()
f1 = theano.function([a1], [a1.shape])

In [ ]:
t1 = np.array([[1,2,3,0,0],[4,5,6,7,0],[2,2,2,2,2]])

In [ ]:
t1.dtype

In [ ]:
f1(t1)

In [ ]:
%reload_ext wordvecs

In [ ]:
np.zeros(10)